In [10]:
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import KFold
from surprise import SlopeOne
from surprise import accuracy
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle as pickle

In [11]:
ratings_data = pd.read_csv("ratings.csv")
print(ratings_data.head())

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [12]:
movie_data = ratings_data.drop('timestamp', axis=1)
# movie_data = pd.merge(ratings_data, movie_names, on='movieId')
movie_data.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [18]:
print(movie_data.loc[(movie_data['userId']==200) & (movie_data['movieId']==100)])
print(movie_data.loc[movie_data['userId']==200,'rating'].mean())
print(movie_data.loc[movie_data['movieId']==100,'rating'].mean())

Empty DataFrame
Columns: [userId, movieId, rating]
Index: []
3.809880239520958
2.7857142857142856


In [4]:
# import sys
# !{sys.executable} -m pip install surprise

In [5]:
reader = Reader(line_format= 'user item rating timestamp',sep = ',',skip_lines=1)
data = Dataset.load_from_file('./ratings.csv',reader = reader)
trainset = data.build_full_trainset()

In [7]:
#Use SlopeOne algorithm
algo = SlopeOne()
algo.fit(trainset)

In [28]:
#Perform scoring predictions on specified users and products
uid =str(200)
iid = str(100)
pred = algo.predict(uid,iid)

In [29]:
print(pred.est)

3.053398758039476


In [48]:
users_ratings_prediction = pd.DataFrame([],columns=['userId','movieId','rating'])
users = pd.Series(pd.unique(movie_data['userId']))
for _,user in tqdm(users.iteritems()):
    not_user_movies = pd.Series(pd.unique(movie_data.loc[movie_data['userId']!=user,'movieId']))
    user_predictions = not_user_movies.apply(lambda movie: algo.predict(str(user),str(movie)).est)
    df = pd.DataFrame(columns = ['movieId', 'rating'])
    df['movieId']=not_user_movies
    df['rating']=user_predictions
    df['userId']=user
    users_ratings_prediction = pd.concat([users_ratings_prediction, df],ignore_index=True)
#     print(users_ratings_prediction.tail())
#     df = pd.DataFrame([not_user_movies,user_predictions], columns=['movieId','rating'])
#     df['userId']=user
#     print(df.tail())
#     for _,movie in not_user_movies.iteritems():
#         pred = algo.predict(str(user),str(movie))
#         users_ratings_prediction.loc[users_ratings_prediction.shape[0]] = [user, movie, pred.est]

610it [04:35,  2.22it/s]


In [49]:
users_ratings_prediction

,userId,movieId,rating
0,1,318,5.000000
1,1,333,4.332579
2,1,1704,4.919075
3,1,3578,4.647980
4,1,6874,4.961733
...,...,...,...
5928189,610,7377,3.191934
5928190,610,8667,3.191934
5928191,610,32302,4.191934
5928192,610,51903,2.191934


In [53]:
users_ratings_prediction.to_csv('calculated_predictions_slope_one.csv',index=False)

In [54]:
users_ratings_prediction = pd.read_csv("calculated_predictions_slope_one.csv")

In [55]:
users_ratings_prediction

,userId,movieId,rating
0,1,318,5.000000
1,1,333,4.332579
2,1,1704,4.919075
3,1,3578,4.647980
4,1,6874,4.961733
...,...,...,...
5928189,610,7377,3.191934
5928190,610,8667,3.191934
5928191,610,32302,4.191934
5928192,610,51903,2.191934


In [63]:
def get_recs(userId, n=10):
    return users_ratings_prediction.loc[users_ratings_prediction['userId']==userId,['movieId','rating']].sort_values(by='rating', ascending=False)[:n]

In [64]:
# получить пять рекомендация для пользователя с id==2
get_recs(2,5)

,movieId,rating
13644,126921,5.0
13712,149350,5.0
13678,136341,5.0
19086,1349,5.0
19085,1310,5.0
